# CD Directory

要求实现cd(current_dir, new dir), 返回最终的path, 比如：
cd(/foo/bar, baz) = /foo/bar/baz
cd(/foo/../, ./baz) = /baz
cd(/, foo/bar/../../baz) = /baz
cd(/, ..) = Null

In [87]:
def cd(cur_dir, new_dir):
    path = cur_dir + '/' + new_dir
    if not path: return '/'

    path_stack = []
    path_tokens = path.split('/')
    for token in path_tokens:
        if token == '..' and path_stack:
            path_stack.pop()
        elif not token or token == '.':
            continue
        elif token == '..' and not path_stack:
            return 'NULL'
        else:
            path_stack.append(token)
    return '/' + '/'.join(path_stack)

In [88]:
# Test cases
assert cd('/a/b/c', 'd') == '/a/b/c/d'
assert cd('/a/b/c', '../d') == '/a/b/d'
assert cd('/a/b/c', '../../d') == '/a/d'
assert cd('/', 'a/b/../../c') == '/c'
assert cd('', '..') == 'NULL'

print("All test cases passed!")

All test cases passed!


第二问可不可以加上对～符号也就是home directory的支持
完成以后难度加大，第三个参数是soft link的dictionary，比如：
cd(/foo/bar, baz, {/foo/bar: /abc}) = /abc/baz
cd(/foo/bar, baz, {/foo/bar: /abc, /abc: /bcd, /bcd/baz: /xyz}) = /xyz
dictionary 里有可能有短匹配和长匹配，应该先匹配长的(more specific), 比如：
cd(/foo/bar, baz, {/foo/bar: /abc, /foo/bar/baz: /xyz}) = /xyz

要判断dictionary里是否有循环

'USE trie tree traverse through the symlink build the tree'
'each walk through will the O(n) n as length of input path'


e.g. for a path of `a/b/c/d/e`, one will try to match with the following order: `a/b/c/d/e`, `a/b/c/d`, `a/b/c`, `a/b`, `a`.

In [1]:
import collections

class TrieNode:
    def __init__(self):
        self.match = None
        self.next = collections.defaultdict(TrieNode)

class DirectoryMatcher:
    def __init__(self):
        self.root = TrieNode()

    def add(self, path, value):
        node = self.root
        for part in path.split('/'):
            if part:
                node = node.next[part]
        node.match = value

    def match(self, path):
        node = self.root
        current_path = ['']
        matched = None
        for part in path.split('/'):
            if not part:
                continue
            if part in node.next:
                node = node.next[part]
                current_path.append(part)
                if node.match is not None:
                    matched = node.match
            else:
                break
        if matched is None:
            return None, None
        return path.replace('/'.join(current_path), matched, 1), matched

def cd(cur_dir, new_dir, dict_to_match):
    visited = set()
    path = cur_dir + '/' + new_dir

    directory_matcher = DirectoryMatcher()
    for key, value in dict_to_match.items():
        directory_matcher.add(key, value)
    while path:
        # print("path: " + path)
        if path in visited:
            raise ValueError("Loop detected in path")
        visited.add(path)
        match, match_part = directory_matcher.match(path)
        print("match: " + str(match))
        if match is None:
            return path
        if match.startswith(path):
            raise ValueError("Loop detected in path")
        if match_part in visited:
            raise ValueError("Loop detected in path")
        path = match

In [2]:
# Test cases

import unittest

class MyTestCase(unittest.TestCase):
    def test_cd(self):
        self.assertEqual(cd("/foo/bar", "baz", {"/foo/bar": "/abc"}), '/abc/baz')
        self.assertEqual(cd("/foo/bar", "baz", {"/foo/bar": "/abc", "/abc": "/bcd", "/bcd/baz": "/xyz"}), '/xyz')
        self.assertEqual(cd("/foo/bar", "baz", {"/foo/bar": "/abc", "/foo/bar/baz": "/xyz"}), '/xyz')
        self.assertRaises(ValueError, cd, "/foo/bar", "baz", {"/foo/bar": "/abc", "/abc": "/foo/bar"})
        self.assertRaises(ValueError, cd, "/foo/bar", "baz", {"/foo/bar": "/abc", "/abc": "/foo/bar/baz"})

tc = MyTestCase()
tc.test_cd()
print("All test cases passed!")

match: /abc/baz
match: None
match: /abc/baz
match: /bcd/baz
match: /xyz
match: None
match: /xyz
match: None
match: /abc/baz
match: /foo/bar/baz
match: /abc/baz
match: /foo/bar/baz/baz
All test cases passed!
